# Bibliotecas

In [1]:
from constraint import Problem, AllDifferentConstraint
import json

# Leitura de Dataset

## Função para ler Dataset apartir de ficheiro texto

In [2]:
def lerDados(filename):
    dados = {"cc" : {}, "olw" : [], "dsd": {}, "tr" : {}, "rr" : {}, "oc" : {}}
    secao = None

    with open(filename, "r", encoding="utf-8") as file:
        for linha in file:
            linha = linha.strip()
            if not linha or linha.startswith("—"):
                continue

            if linha.startswith("#"):
                secao = linha.split()[0][1:]
                continue

            partes = linha.split()

            match secao:
                case "cc":
                    turma, cursos = partes[0], partes[1:]
                    dados["cc"][turma] = cursos
                
                case "olw":
                    if partes:
                        dados["olw"].append(partes[0])
                
                case "dsd":
                    docente, cursos = partes[0], partes[1:]
                    dados["dsd"][docente] = cursos

                case "tr":
                    docente, restricoes = partes[0], list(map(int, partes[1:]))
                    dados["tr"][docente] = restricoes

                case "rr":
                    curso, sala = partes[0], partes[1]
                    dados["rr"][curso] = sala

                case "oc":
                    curso, idx = partes[0], int(partes[1])
                    dados["oc"][curso] = idx

                case _:
                    pass

    return dados

## Leitura de dados e inicialiação de CSP Problem

In [3]:

dados = lerDados("ClassTT_01_tiny.txt")
print("A carregar dados")
print(dados)

# Inicializar problema CSP
problem = Problem()
blocos = list(range(1, 21))  # 20 blocos semanais

A carregar dados
{'cc': {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'], 't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'], 't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']}, 'olw': [], 'dsd': {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}, 'tr': {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}, 'rr': {'UC14': 'Lab01', 'UC22': 'Lab01'}, 'oc': {'UC21': 2, 'UC31': 2}}


# Criação de variáveis

## Criação de aulas para cada UC
Owl - one weekly lesson
O nome das variáveis é a UC + o número da sua lição semanal
Exemplo: UC11_1 e UC11_2, é uma UC não presente em OWL logo tem 2 aulas semanais


## Restrição de blocos

A restrição (constraint) aplicada durante a criação das variáveis tem como objetivo garantir que 2 blocos da memsa UC (unidade curricular), X e Y, ficam em blocos distintos
Várias aulas podem ocorrer em simultanêo mas todas de UCs diferentes

In [4]:
for turma, cursos in dados["cc"].items():
    for curso in cursos:
        if curso not in dados.get("olw", []):
            problem.addVariable(f"{curso}_1", blocos)
            problem.addVariable(f"{curso}_2", blocos)
            problem.addConstraint(lambda x, y: x != y, (f"{curso}_1", f"{curso}_2")) # Garantir que as duas aulas são em blocos diferentes
        else:
            problem.addVariable(f"{curso}_1", blocos) #UC Owls têm apenas 1 aula por semana

# Criação de restrições (Constraints)

* Restrição de bloco de cada docente
* Restrição de tempo dos docentes
* Retrição de blocos por turma
* Restrição de salas

## Restrição de bloco de cada docente
Mesmo docente não pode dar mais que uma aulas no mesmo bloco

In [5]:
for docente, cursos in dados["dsd"].items():
    variaveis = []
    for curso in cursos:    #array com todas UCs de um docente UC_1 e UC_2 ou apenas UC_1 se for owl
        if curso not in dados.get("olw", []):
            variaveis.extend([f"{curso}_1", f"{curso}_2"])
        else:
            variaveis.append(f"{curso}_1")
    problem.addConstraint(AllDifferentConstraint(), variaveis) # Constraint para que não tenha UCs repetidas


## Restrição de tempo dos docentes
Docentes contêm restrições de tempo, um docente pode indicar que não dá aulas nos blocos [ X, Y, Z, etc...]

In [6]:
for docente, indisponiveis in dados.get("tr", {}).items():
    cursos = dados["dsd"].get(docente, []) #UCs do docente
    for curso in cursos:
        if curso not in dados.get("olw", []):
            for i in [1, 2]:
                problem.addConstraint(
                    lambda bloco, indisponiveis=indisponiveis: bloco not in indisponiveis, #Bloco = variável , indisponiveis = lista de blocos indisponiveis (Dominio da variável)
                    (f"{curso}_{i}",)
                )
        else:
            problem.addConstraint(
                lambda bloco, indisponiveis=indisponiveis: bloco not in indisponiveis, #Compara UC_x com blocos indisponiveis para o docente que leciona a UC
                (f"{curso}_1",)
            )

## Retrição de blocos por turma
Uma turma não pode ter mais que uma aula no mesmo bloco

In [7]:
for turma, cursos in dados["cc"].items():
    variaveis = []
    for curso in cursos:    #array com todas UCs de uma turma UC_1 e UC_2 ou apenas UC_1 se for owl
        if curso not in dados.get("olw", []):
            variaveis.extend([f"{curso}_1", f"{curso}_2"])
        else:
            variaveis.append(f"{curso}_1")
    problem.addConstraint(AllDifferentConstraint(), variaveis) # Constraint para que não tenha UCs repetidas

## Restrição de salas
Certas UCs só podem ser lecionadas em certas salas, logo não é pode ter mais que uma UC que necessitem a mesma sala no mesmo bloco

In [8]:
### Criar dicionário para salas atribuídas

In [9]:
salas = {}

for curso, sala in dados.get("rr", {}).items():
    if sala not in salas:
        salas[sala] = []
    if curso not in dados.get("olw", []):
        salas[sala].extend([f"{curso}_1", f"{curso}_2"])
    else:
        salas[sala].append(f"{curso}_1")


### Aplicar restrição nos blocos que usam as salas no dicionário

In [10]:

# Cada sala só pode ter uma aula por bloco
for sala, variaveis in salas.items():
    problem.addConstraint(AllDifferentConstraint(), variaveis) # Constraint para que não tenha UCs com a mesma sala não tenham o mesmo bloco

# Solução
Mostra a primeira solução encontrada para o CSP e mostra os resultados em formato JSON para que seja mais legível

In [11]:

solucao = problem.getSolution()

if solucao:
    print("\nSolução encontrada:")
    print(json.dumps(solucao, indent=4))
else:
    print("Nenhuma solução encontrada")



Solução encontrada:
{
    "UC14_1": 20,
    "UC14_2": 19,
    "UC22_1": 18,
    "UC22_2": 17,
    "UC12_1": 12,
    "UC12_2": 11,
    "UC23_1": 10,
    "UC23_2": 9,
    "UC32_1": 8,
    "UC32_2": 7,
    "UC24_1": 16,
    "UC24_2": 15,
    "UC13_1": 18,
    "UC13_2": 17,
    "UC33_1": 14,
    "UC33_2": 13,
    "UC34_1": 16,
    "UC34_2": 15,
    "UC35_1": 6,
    "UC35_2": 5,
    "UC15_1": 14,
    "UC15_2": 13,
    "UC25_1": 8,
    "UC25_2": 7,
    "UC31_1": 20,
    "UC31_2": 19,
    "UC21_1": 14,
    "UC21_2": 13,
    "UC11_1": 16,
    "UC11_2": 15
}
